In [6]:
import itertools
import numpy as np
starts = np.arange(0, 9001, 200)
ends = np.arange(200, 9201, 200)
ends[-1] = 9045
expts = ['baseline_bal', 'reduction', 'adult']
data_index = range(ends.shape[0])
iteration = range(10)
lrs = [1e-2, 1e-3, 4e-4, 1e-4]
a = itertools.product(expts, data_index, iteration, lrs)

In [2]:
b = [i for i in a]

In [3]:
len(b)

5520

In [10]:
5520/3 


1840.0

In [2]:
from scipy.stats import norm
norm.cdf(0)

0.5

In [9]:
enumerate(b)

In [2]:
import itertools
expts = ['reduction', 'baseline_bal', 'sensr'] 
iteration = range(10)
lrs = [1e-2, 1e-3, 4e-4, 1e-4]

a = itertools.product(expts, iteration, lrs)
b = [i for i in a]

In [3]:
len(b)

120

In [1]:
import re
import pandas as pd
with open('summary_same_step.out', 'r') as f:
    data = f.read()
entries = re.split('\n', data)[:-1]
entries[1]

"{'algo': 'baseline_bal', 'seed': (98539, 77708), 'lr': 0.0004, 'accuracy': 0.8117191818684356, 'lb': 1.0585399219951463, 'pval': 1.0, 'bal_acc': 0.8283820254159571, 'gap_rms_gen': 0.14269193469019995, 'mean_gap_gen': 0.13562301671702554, 'max_gap_gen': 0.17997823695326942, 'average_odds_difference_gen': -0.13562301671702554, 'equal_opportunity_difference_gen': -0.09126779648078165, 'statistical_parity_difference_gen': -0.2884567528053568, 'gap_rms_race': 0.046926087936521796, 'mean_gap_race': 0.04322317786320107, 'max_gap_race': 0.06149377239190379, 'average_odds_difference_race': -0.04322317786320107, 'equal_opportunity_difference_race': -0.024952583334498346, 'statistical_parity_difference_race': -0.11265307925306034}"

In [2]:
entries_dict = [eval(e) for e in entries]

In [3]:
data = pd.DataFrame(entries_dict)

In [102]:
lr = 1e-2
data_lr = data.loc[data['lr'] == lr]
data_lr = data_lr.rename(columns= {'average_odds_difference_gen': 'AOD-gen', 'average_odds_difference_race': 'AOD-race', 'statistical_parity_difference_gen': 'SPD-gen', 'statistical_parity_difference_race': 'SPD-race', 'equal_opportunity_difference_race': 'EOD-race', 'equal_opportunity_difference_gen': 'EOD-gen'})

In [103]:
measure = ['bal_acc', 'AOD-gen', 'AOD-race', 'lb', 'pval']
agg_dict = dict()
for key in measure:
    agg_dict[key] = ['mean', 'std']
result = data_lr.groupby(['algo'], as_index=False).agg(agg_dict)

0    0.817$\pm$0.007
1    0.825$\pm$0.003
2    0.800$\pm$0.005
3    0.765$\pm$0.012
dtype: object

In [104]:
rows = ['sensr', 'reduction', 'baseline', 'baseline_bal']
row_names = ['Baseline', 'Project', 'Reduction','SenSR']
colnames=['Balanced Acc', '$\\text{AOD}_{\\text{gen}}$', '$\\text{AOD}_{\\text{race}}$', '$T_n$', 'Rejection Prop']
table = pd.DataFrame(index=row_names)
table

""
SenSR
Reduction
Baseline
Project


In [109]:
for i, c in enumerate(measure):
    if c == 'pval':
        idx_best = (1-result[c]['mean']).idxmin()
        result[colnames[i]] = (1-result[c]['mean']).apply("{:.3f}".format)
        result.at[idx_best, colnames[i]] = '\\textbf{' + result[colnames[i]][idx_best] + '}'
        
    elif c == 'bal_acc':
        idx_best = (result[c]['mean']).idxmax()
        result[colnames[i]] = result[c]['mean'].apply("{:.3f}".format)+ '$\pm$' + result[c]['std'].apply("{:.3f}".format)
        best_val = result[colnames[i]][idx_best].split('$\pm$')
        best_val = '$\pm$'.join(['\\textbf{' + best_val[0] + '}', best_val[1]])
        result.at[idx_best, colnames[i]] = best_val
    elif c == 'lb':
        idx_best = (result[c]['mean']).idxmin()
        result[colnames[i]] = result[c]['mean'].apply("{:.3f}".format)+ '$\pm$' + result[c]['std'].apply("{:.3f}".format)
        best_val = result[colnames[i]][idx_best].split('$\pm$')
        best_val = '$\pm$'.join(['\\textbf{' + best_val[0] + '}', best_val[1]])
        result.at[idx_best, colnames[i]] = best_val
    else:
        idx_best = (result[c]['mean']).abs().idxmin()
        result[colnames[i]] = result[c]['mean'].apply("{:.3f}".format)+ '$\pm$' + result[c]['std'].apply("{:.3f}".format)
        best_val = result[colnames[i]][idx_best].split('$\pm$')
        best_val = '$\pm$'.join(['\\textbf{' + best_val[0] + '}', best_val[1]])
        result.at[idx_best, colnames[i]] = best_val
#print(result.to_latex(escape=False, column_format='l' + 'c'*len(colnames)))
result

algo   bal_acc             AOD-gen            AOD-race            \
                     mean       std      mean       std      mean       std   
0      baseline  0.816889  0.006759 -0.151117  0.026236 -0.060650  0.015300   
1  baseline_bal  0.825290  0.003224 -0.147170  0.014956 -0.052808  0.014611   
2     reduction  0.799689  0.004515  0.001286  0.021245 -0.026839  0.013104   
3         sensr  0.765446  0.011889 -0.073894  0.033192 -0.047882  0.008244   

         lb                pval                      Balanced Acc  \
       mean       std      mean     std                             
0  3.675913  2.163627  0.000000  0.0000           0.817$\pm$0.007   
1  1.660149  0.355157  0.097841  0.3094  \textbf{0.825}$\pm$0.003   
2  5.712395  2.264394  0.000000  0.0000           0.800$\pm$0.005   
3  1.020763  0.008241  1.000000  0.0000           0.765$\pm$0.012   

  $\text{AOD}_{\text{gen}}$ $\text{AOD}_{\text{race}}$  \
                                                         
0          -0.151$\pm$0.026           -0.061$\pm$0.015   
1          -0.147$\pm$0.015           -0.053$\pm$0.015   
2  \textbf{0.001}$\pm$0.021  \textbf{-0.027}$\pm$0.013   
3          -0.074$\pm$0.033           -0.048$\pm$0.008   

                      $T_n$  Rejection Prop  
                                             
0           3.676$\pm$2.164           1.000  
1           1.660$\pm$0.355           0.902  
2           5.712$\pm$2.264           1.000  
3  \textbf{1.021}$\pm$0.008  \textbf{0.000}

In [106]:
ind = dict()
for i, expt in enumerate(row_names):
    ind[i] = expt
    res = result[colnames].rename(index=ind)

In [107]:
result['bal_acc']['mean'].idxmax()

1

In [108]:
res

,Balanced Acc,$\text{AOD}_{\text{gen}}$,$\text{AOD}_{\text{race}}$,$T_n$,Rejection Prop
,,,,,
SenSR,0.817$\pm$0.007,-0.151$\pm$0.026,-0.061$\pm$0.015,3.676$\pm$2.164,1.000
Reduction,\textbf{0.825}$\pm$0.003,-0.147$\pm$0.015,-0.053$\pm$0.015,1.660$\pm$0.355,0.902
Baseline,0.800$\pm$0.005,\textbf{0.001}$\pm$0.021,\textbf{-0.027}$\pm$0.013,5.712$\pm$2.264,1.000
Project,0.765$\pm$0.012,-0.074$\pm$0.033,-0.048$\pm$0.008,\textbf{1.021}$\pm$0.008,\textbf{0.000}


In [101]:
print(res.to_latex(escape=False, column_format='l' + 'c'*len(colnames)))

\begin{tabular}{lccccc}
\toprule
{} &              Balanced Acc &  $\text{AOD}_{\text{gen}}$ & $\text{AOD}_{\text{race}}$ &                     $T_n$ &  Rejection Prop \\
{} \\
\midrule
SenSR     &           0.817$\pm$0.007 &  \textbf{-0.151}$\pm$0.026 &  \textbf{-0.061}$\pm$0.015 &           3.676$\pm$2.164 &           1.000 \\
Reduction &  \textbf{0.825}$\pm$0.003 &           -0.147$\pm$0.015 &           -0.053$\pm$0.015 &           1.660$\pm$0.355 &           0.902 \\
Baseline  &           0.800$\pm$0.005 &            0.001$\pm$0.021 &           -0.027$\pm$0.013 &           5.712$\pm$2.264 &           1.000 \\
Project   &           0.765$\pm$0.012 &           -0.074$\pm$0.033 &           -0.048$\pm$0.008 &  \textbf{1.021}$\pm$0.008 &  \textbf{0.000} \\
\bottomrule
\end{tabular}

